In [1]:
from transformers import pipeline
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
)

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-05-23 20:09:51.795441: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-23 20:09:51.828924: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 20:09:51.828953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 20:09:51.829743: E external/local_xla/xla/stream_executor/cuda/c

In [2]:
# Load the tokenizer and model
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


# Define the preprocess function
def preprocess_function(examples):
    inputs = examples["text"]
    inputs = [str(i) for i in inputs]  # Ensure inputs are strings
    model_inputs = tokenizer(
        inputs, max_length=512, truncation=True, padding="max_length"
    )

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [str(i) for i in examples["headline"]],
            max_length=150,
            truncation=True,
            padding="max_length",
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Apply the preprocess function to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

NameError: name 'dataset' is not defined

In [ ]:
# Use a DataCollator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Set training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Increased batch size
    per_device_eval_batch_size=8,  # Increased batch size
    gradient_accumulation_steps=4,  # Use gradient accumulation
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    bf16=True,  # Use mixed precision training if supported
    predict_with_generate=True,
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Test the model
text = "Your input text here..."
inputs = tokenizer(
    text, return_tensors="pt", max_length=512, truncation=True, padding="max_length"
).to(device)
summary_ids = model.generate(
    inputs.input_ids,
    max_new_tokens=150,  # Use max_new_tokens instead of max_length
    min_length=40,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True,
)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)